# Language Translation

In this notebook, we will train a sequence-to-sequence model on a dataset of English and French sentences that can translate new sentences from English to French.

We shall train with a small portion of the English and French corpus since training the entire corpus will require a long time.

* [Load and Explore the Data](#part1)
* [Preprocess the Data and Save](#part2)
* [Create the Neural Network](#part3)
* [Train the Neural Network](#part4)
* [Test the Sentence to Sequence Model](#part5)

In [ ]:
# Import required libraries

import numpy as np
import helper

### Check the Version of TensorFlow and Access to GPU

In [11]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0


/home/kokmeng/anaconda3/envs/tf13/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # This is added back by InteractiveShellApp.init_path()


---
<a id='part1'></a>

## Load and Explore the Data

In [1]:
source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

In [2]:
view_sentence_range = (0, 10)

print('Approximate number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

---
<a id='part2'></a>

## Preprocess the Data and Save

In [ ]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids. The <EOS> word id added at the end of target_text to help
    the neural network to predict when the sentence should end.
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_sentences = source_text.lower().split('\n')
    source_text_split = [sentence.split() for sentence in source_sentences]
    source_ids = [[source_vocab_to_int[word] for word in sentence] for sentence in source_text_split]
    target_sentences = target_text.lower().split('\n')
    target_text_split = [sentence.split() + ['<EOS>'] for sentence in target_sentences]
    target_ids = [[target_vocab_to_int[word] for word in sentence] for sentence in target_text_split]
    return source_ids, target_ids

helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point

In [5]:
import numpy as np
import helper
import tensorflow as tf
from tensorflow.python.layers.core import Dense

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

---
<a id='part3'></a>

## Create the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input

In [1]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, [], name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    return inputs, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding by removing the last word id from each batch in target_data 
    and concat the GO ID to the beginning of each batch.
    :param target_data: Target Placeholder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    processed_data = tf.concat([go_id, target_data[:,:-1]], axis=1)
    return processed_data

### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [4]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Embed the encoder input
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size=source_vocab_size,
                                             embed_dim=encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size, keep_prob):
        # LSTM cell
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])

    # Pass cell and embedded input to tf.nn.dynamic.rnn
    output, final_state = tf.nn.dynamic_rnn(cell, embed, sequence_length=source_sequence_length, dtype=tf.float32)
    return output, final_state

### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [5]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
   :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    basic_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder, 
                                                                   maximum_iterations=max_summary_length)
    return basic_decoder_output

### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS ID
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), 
                           [batch_size], name='start_tokens')
    training_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    inference_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder, 
                                                                       maximum_iterations=max_target_sequence_length)
    return inference_decoder_output

### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [7]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size, keep_prob):
        # LSTM cell
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])

    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    
    with tf.variable_scope('decode'):
        tr_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                                                 target_sequence_length, max_target_sequence_length, 
                                                 output_layer, keep_prob)
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']
        inf_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                                  end_of_sequence_id, max_target_sequence_length,
                                                  target_vocab_size, output_layer, batch_size, keep_prob)
    return tr_decoder_output, inf_decoder_output

### Build the Neural Network
Apply the functions implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [8]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,  
                                  source_sequence_length, source_vocab_size, 
                                  enc_embedding_size)
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    tr_decoder_output, inf_decoder_output = decoding_layer(dec_input, enc_state, target_sequence_length, 
                                                           max_target_sentence_length, rnn_size, num_layers, 
                                                           target_vocab_to_int, target_vocab_size, 
                                                           batch_size, keep_prob, 
                                                           dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output

---
<a id='part4'></a>

## Train the Neural Network

### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.55
display_step = True

### Build the Graph

In [15]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [9]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

### Split data to training and validation sets

In [ ]:
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = \
    next(get_batches(valid_source,
                     valid_target,
                     batch_size,
                     source_vocab_to_int['<PAD>'],
                     target_vocab_to_int['<PAD>']))

### Train

In [17]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))


with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2336, Validation Accuracy: 0.3100, Loss: 5.7067
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 5.5046
Epoch   0 Batch    3/269 - Train Accuracy: 0.2444, Validation Accuracy: 0.3096, Loss: 5.3443
Epoch   0 Batch    4/269 - Train Accuracy: 0.2317, Validation Accuracy: 0.3096, Loss: 5.1667
Epoch   0 Batch    5/269 - Train Accuracy: 0.2325, Validation Accuracy: 0.3096, Loss: 4.9802
Epoch   0 Batch    6/269 - Train Accuracy: 0.2786, Validation Accuracy: 0.3098, Loss: 4.6574
Epoch   0 Batch    7/269 - Train Accuracy: 0.2865, Validation Accuracy: 0.3210, Loss: 4.5111
Epoch   0 Batch    8/269 - Train Accuracy: 0.2535, Validation Accuracy: 0.3224, Loss: 4.5242
Epoch   0 Batch    9/269 - Train Accuracy: 0.2923, Validation Accuracy: 0.3328, Loss: 4.2981
Epoch   0 Batch   10/269 - Train Accuracy: 0.2673, Validation Accuracy: 0.3407, Loss: 4.3466
Epoch   0 Batch   11/269 - Train Accuracy: 0.3061, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4313, Validation Accuracy: 0.4840, Loss: 2.6237
Epoch   0 Batch   91/269 - Train Accuracy: 0.4654, Validation Accuracy: 0.4876, Loss: 2.4696
Epoch   0 Batch   92/269 - Train Accuracy: 0.4648, Validation Accuracy: 0.4900, Loss: 2.4512
Epoch   0 Batch   93/269 - Train Accuracy: 0.4806, Validation Accuracy: 0.4850, Loss: 2.3671
Epoch   0 Batch   94/269 - Train Accuracy: 0.4695, Validation Accuracy: 0.4847, Loss: 2.4391
Epoch   0 Batch   95/269 - Train Accuracy: 0.4676, Validation Accuracy: 0.4904, Loss: 2.4305
Epoch   0 Batch   96/269 - Train Accuracy: 0.4682, Validation Accuracy: 0.4932, Loss: 2.4399
Epoch   0 Batch   97/269 - Train Accuracy: 0.4639, Validation Accuracy: 0.4831, Loss: 2.4142
Epoch   0 Batch   98/269 - Train Accuracy: 0.4752, Validation Accuracy: 0.4882, Loss: 2.3936
Epoch   0 Batch   99/269 - Train Accuracy: 0.4394, Validation Accuracy: 0.4923, Loss: 2.5347
Epoch   0 Batch  100/269 - Train Accuracy: 0.4892, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.4940, Validation Accuracy: 0.5149, Loss: 1.8394
Epoch   0 Batch  180/269 - Train Accuracy: 0.4895, Validation Accuracy: 0.5150, Loss: 1.8077
Epoch   0 Batch  181/269 - Train Accuracy: 0.4605, Validation Accuracy: 0.4867, Loss: 1.8006
Epoch   0 Batch  182/269 - Train Accuracy: 0.4940, Validation Accuracy: 0.5104, Loss: 1.8258
Epoch   0 Batch  183/269 - Train Accuracy: 0.5521, Validation Accuracy: 0.5088, Loss: 1.5729
Epoch   0 Batch  184/269 - Train Accuracy: 0.4412, Validation Accuracy: 0.4950, Loss: 1.8741
Epoch   0 Batch  185/269 - Train Accuracy: 0.4962, Validation Accuracy: 0.5067, Loss: 1.7805
Epoch   0 Batch  186/269 - Train Accuracy: 0.4607, Validation Accuracy: 0.5118, Loss: 1.8602
Epoch   0 Batch  187/269 - Train Accuracy: 0.4807, Validation Accuracy: 0.5038, Loss: 1.7594
Epoch   0 Batch  188/269 - Train Accuracy: 0.4769, Validation Accuracy: 0.4907, Loss: 1.7193
Epoch   0 Batch  189/269 - Train Accuracy: 0.4965, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.4578, Validation Accuracy: 0.5057, Loss: 1.5066
Epoch   1 Batch    2/269 - Train Accuracy: 0.4964, Validation Accuracy: 0.5347, Loss: 1.4761
Epoch   1 Batch    3/269 - Train Accuracy: 0.4622, Validation Accuracy: 0.5077, Loss: 1.5123
Epoch   1 Batch    4/269 - Train Accuracy: 0.4548, Validation Accuracy: 0.5157, Loss: 1.4999
Epoch   1 Batch    5/269 - Train Accuracy: 0.4566, Validation Accuracy: 0.5173, Loss: 1.5131
Epoch   1 Batch    6/269 - Train Accuracy: 0.4974, Validation Accuracy: 0.5133, Loss: 1.3768
Epoch   1 Batch    7/269 - Train Accuracy: 0.4862, Validation Accuracy: 0.5036, Loss: 1.4139
Epoch   1 Batch    8/269 - Train Accuracy: 0.4558, Validation Accuracy: 0.5100, Loss: 1.4799
Epoch   1 Batch    9/269 - Train Accuracy: 0.4725, Validation Accuracy: 0.5118, Loss: 1.4352
Epoch   1 Batch   10/269 - Train Accuracy: 0.4467, Validation Accuracy: 0.4972, Loss: 1.4603
Epoch   1 Batch   11/269 - Train Accuracy: 0.4813, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.4832, Validation Accuracy: 0.5376, Loss: 1.2655
Epoch   1 Batch   91/269 - Train Accuracy: 0.5240, Validation Accuracy: 0.5502, Loss: 1.1860
Epoch   1 Batch   92/269 - Train Accuracy: 0.5405, Validation Accuracy: 0.5566, Loss: 1.1836
Epoch   1 Batch   93/269 - Train Accuracy: 0.5502, Validation Accuracy: 0.5568, Loss: 1.1440
Epoch   1 Batch   94/269 - Train Accuracy: 0.5437, Validation Accuracy: 0.5613, Loss: 1.2100
Epoch   1 Batch   95/269 - Train Accuracy: 0.5390, Validation Accuracy: 0.5636, Loss: 1.1849
Epoch   1 Batch   96/269 - Train Accuracy: 0.5293, Validation Accuracy: 0.5455, Loss: 1.1664
Epoch   1 Batch   97/269 - Train Accuracy: 0.5294, Validation Accuracy: 0.5557, Loss: 1.1894
Epoch   1 Batch   98/269 - Train Accuracy: 0.5667, Validation Accuracy: 0.5688, Loss: 1.1611
Epoch   1 Batch   99/269 - Train Accuracy: 0.5215, Validation Accuracy: 0.5630, Loss: 1.2470
Epoch   1 Batch  100/269 - Train Accuracy: 0.5478, Validation Accuracy

...

Epoch   8 Batch  179/269 - Train Accuracy: 0.8542, Validation Accuracy: 0.8671, Loss: 0.2066
Epoch   8 Batch  180/269 - Train Accuracy: 0.8746, Validation Accuracy: 0.8673, Loss: 0.2052
Epoch   8 Batch  181/269 - Train Accuracy: 0.8597, Validation Accuracy: 0.8661, Loss: 0.2175
Epoch   8 Batch  182/269 - Train Accuracy: 0.8831, Validation Accuracy: 0.8584, Loss: 0.2046
Epoch   8 Batch  183/269 - Train Accuracy: 0.8920, Validation Accuracy: 0.8675, Loss: 0.1776
Epoch   8 Batch  184/269 - Train Accuracy: 0.8725, Validation Accuracy: 0.8746, Loss: 0.2105
Epoch   8 Batch  185/269 - Train Accuracy: 0.8875, Validation Accuracy: 0.8697, Loss: 0.2022
Epoch   8 Batch  186/269 - Train Accuracy: 0.8696, Validation Accuracy: 0.8722, Loss: 0.2034
Epoch   8 Batch  187/269 - Train Accuracy: 0.8721, Validation Accuracy: 0.8716, Loss: 0.2011
Epoch   8 Batch  188/269 - Train Accuracy: 0.8711, Validation Accuracy: 0.8673, Loss: 0.1989
Epoch   8 Batch  189/269 - Train Accuracy: 0.8756, Validation Accuracy

Epoch   9 Batch    1/269 - Train Accuracy: 0.8855, Validation Accuracy: 0.8709, Loss: 0.1933
Epoch   9 Batch    2/269 - Train Accuracy: 0.8742, Validation Accuracy: 0.8751, Loss: 0.1970
Epoch   9 Batch    3/269 - Train Accuracy: 0.8872, Validation Accuracy: 0.8753, Loss: 0.2040
Epoch   9 Batch    4/269 - Train Accuracy: 0.8487, Validation Accuracy: 0.8612, Loss: 0.2034
Epoch   9 Batch    5/269 - Train Accuracy: 0.8722, Validation Accuracy: 0.8815, Loss: 0.2076
Epoch   9 Batch    6/269 - Train Accuracy: 0.8948, Validation Accuracy: 0.8750, Loss: 0.1890
Epoch   9 Batch    7/269 - Train Accuracy: 0.8842, Validation Accuracy: 0.8737, Loss: 0.1867
Epoch   9 Batch    8/269 - Train Accuracy: 0.8889, Validation Accuracy: 0.8833, Loss: 0.1983
Epoch   9 Batch    9/269 - Train Accuracy: 0.8826, Validation Accuracy: 0.8818, Loss: 0.2028
Epoch   9 Batch   10/269 - Train Accuracy: 0.8878, Validation Accuracy: 0.8778, Loss: 0.1874
Epoch   9 Batch   11/269 - Train Accuracy: 0.8987, Validation Accuracy

Epoch   9 Batch   90/269 - Train Accuracy: 0.8724, Validation Accuracy: 0.8679, Loss: 0.1891
Epoch   9 Batch   91/269 - Train Accuracy: 0.8966, Validation Accuracy: 0.8730, Loss: 0.1723
Epoch   9 Batch   92/269 - Train Accuracy: 0.8931, Validation Accuracy: 0.8748, Loss: 0.1731
Epoch   9 Batch   93/269 - Train Accuracy: 0.8814, Validation Accuracy: 0.8711, Loss: 0.1749
Epoch   9 Batch   94/269 - Train Accuracy: 0.8760, Validation Accuracy: 0.8714, Loss: 0.1974
Epoch   9 Batch   95/269 - Train Accuracy: 0.8814, Validation Accuracy: 0.8764, Loss: 0.1785
Epoch   9 Batch   96/269 - Train Accuracy: 0.8687, Validation Accuracy: 0.8713, Loss: 0.1784
Epoch   9 Batch   97/269 - Train Accuracy: 0.8894, Validation Accuracy: 0.8761, Loss: 0.1724
Epoch   9 Batch   98/269 - Train Accuracy: 0.8968, Validation Accuracy: 0.8728, Loss: 0.1768
Epoch   9 Batch   99/269 - Train Accuracy: 0.8787, Validation Accuracy: 0.8722, Loss: 0.1768
Epoch   9 Batch  100/269 - Train Accuracy: 0.8988, Validation Accuracy

Epoch   9 Batch  179/269 - Train Accuracy: 0.8810, Validation Accuracy: 0.8824, Loss: 0.1756
Epoch   9 Batch  180/269 - Train Accuracy: 0.8879, Validation Accuracy: 0.8794, Loss: 0.1646
Epoch   9 Batch  181/269 - Train Accuracy: 0.8749, Validation Accuracy: 0.8802, Loss: 0.1798
Epoch   9 Batch  182/269 - Train Accuracy: 0.8948, Validation Accuracy: 0.8835, Loss: 0.1736
Epoch   9 Batch  183/269 - Train Accuracy: 0.8991, Validation Accuracy: 0.8846, Loss: 0.1465
Epoch   9 Batch  184/269 - Train Accuracy: 0.8846, Validation Accuracy: 0.8852, Loss: 0.1715
Epoch   9 Batch  185/269 - Train Accuracy: 0.9008, Validation Accuracy: 0.8809, Loss: 0.1716
Epoch   9 Batch  186/269 - Train Accuracy: 0.8862, Validation Accuracy: 0.8830, Loss: 0.1714
Epoch   9 Batch  187/269 - Train Accuracy: 0.8880, Validation Accuracy: 0.8781, Loss: 0.1684
Epoch   9 Batch  188/269 - Train Accuracy: 0.9009, Validation Accuracy: 0.8875, Loss: 0.1709
Epoch   9 Batch  189/269 - Train Accuracy: 0.9038, Validation Accuracy

Model Trained and Saved


### Save Parameters

In [18]:
helper.save_params(save_path)

# Checkpoint

In [17]:
import tensorflow as tf
import numpy as np
import helper

batch_size = 512

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

---
<a id='part5'></a>

## Test the Sentence to Sequence Model

In [15]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    UNK_id = vocab_to_int['<UNK>']
    seq = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            seq.append(vocab_to_int[word])
        else:
            seq.append(UNK_id)
    return seq

## Translate from English to French

In [18]:
translate_sentence = 'he saw a old yellow truck .'

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [130, 190, 107, 91, 99, 127, 44]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [327, 260, 347, 222, 229, 130, 16, 1]
  French Words: il conduit ce vieux camion jaune . <EOS>


## Conclusion

Since the dataset only has a vocabulary of 227 English words, the translation is only good for limited number of sentences. To create a better translation model, we can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar). This dataset has more vocabulary and richer in topics discussed but requires a much longer time to train.